In [1]:
%matplotlib inline

import os
import glob
import imageio

from math import atan2, degrees

import numpy as np
import pandas as pd

import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import cm
from matplotlib.colors import ListedColormap

from scipy.ndimage import gaussian_filter

# Plot hetmaps over selected frame without data from rejected sessions.

In [2]:
DISPLAY_SIZE = (320,200)
extent = [0, DISPLAY_SIZE[0], DISPLAY_SIZE[1], 0]

cmap = cm.jet
my_cmap = cmap(np.arange(cmap.N))
# Set alpha
my_cmap[:,-1] = np.linspace(0, 1, cmap.N)
# Create new colormap
cm_jet = ListedColormap(my_cmap)

def visual_span_from_distance(distance, scale=1):
    h = 324 / scale # Monitor height in cm
    d = distance / scale # Distance between monitor and participant in cm
    r = 800 / scale # Vertical resolution of the monitor
    size_in_deg = 5. # The stimulus size in pixels
    # Calculate the number of degrees that correspond to a single pixel. This will
    # generally be a very small value, something like 0.03.
    deg_per_px = degrees(atan2(.5*h, d)) / (.5*r)
    # Calculate the size of the stimulus in degrees
    size_in_px = size_in_deg / deg_per_px
    return size_in_px

def normalize(data):
    return ( data - data.min() ) / ( data.max() - data.min() )

def get_data(mediafile):
    media = imageio.get_reader("../Dataset/Mahnob/data/media_24/{}.avi".format(mediafile))
    coordinates = pd.DataFrame.from_csv("sm_data_coord_{}.csv".format(mediafile)).applymap(eval)
    fixations = pd.DataFrame.from_csv("sm_data_fix_{}.csv".format(mediafile)).applymap(eval)
    return media, coordinates, fixations

def get_media_frame(media, frames):
    images = []
    for frame in frames:
        media.set_image_index(frame-1)
        images.append(media.get_next_data())
    return images

def get_frame_coordinates(coord, frames):
    frame_coordinates = []
    for frame in frames:
        frame_coordinates.append(coord.T[frame-1])
    return frame_coordinates

def get_frame_fixations(fix, frames):
    frame_fixations = []
    for frame in frames:
        frame_fixations.append(fix.T[frame-1])
    return frame_fixations

def create_image_from_fixation(x, y, v, distance=None):
    m = np.zeros([DISPLAY_SIZE[1], DISPLAY_SIZE[0]], dtype=float)
    m[int(y), int(x)] = v
    
    if distance is None:
        sigma = 12
    else:
        sigma = 0.17 * visual_span_from_distance(distance, scale=4)
    
    m = gaussian_filter(m, sigma=sigma)
    return m

def compute_frame_saliency_map(data):
    ## matrix of zeros
    M = np.zeros([DISPLAY_SIZE[1], DISPLAY_SIZE[0]], dtype=float)
    # gaussian matrix
    for x, y, duration, distance, _ in data:
        if 0<=x<=DISPLAY_SIZE[0] and 0<=y<=DISPLAY_SIZE[1]:
            m = create_image_from_fixation(x, y, v=duration, distance=distance)
            M = np.add(M, m)
    return M

def compute_frame_emotion_saliency_map(data):
    ## matrix of zeros
    M = np.zeros([DISPLAY_SIZE[1], DISPLAY_SIZE[0]], dtype=float)
    # gaussian matrix
    for x, y, duration, distance, feature in data:
        if 0<=x<=DISPLAY_SIZE[0] and 0<=y<=DISPLAY_SIZE[1]:
            m = create_image_from_fixation(x, y, v=feature, distance=distance)
            M = np.add(M, m)
    return M

def get_frame_saliency_maps(gaze_data, frames):
    frame_saliency_maps = []
    
    for frame in frames:
        gd = gaze_data.T[frame]
        frame_saliency_maps.append(compute_frame_saliency_map(gd))
        
    frame_saliency_maps = np.array(frame_saliency_maps)
    
    # apply normalization to get the information of the relation with other heatmaps
    frame_saliency_maps = normalize(frame_saliency_maps)
    
    # apply some threashold to remove usesell peak
    frame_saliency_maps[frame_saliency_maps<=.2] = 0
    
    return frame_saliency_maps

def get_frame_emotion_saliency_maps(gaze_data, frames):
    frame_saliency_maps = []
    
    for frame in frames:
        gd = gaze_data.T[frame]
        frame_saliency_maps.append(compute_frame_emotion_saliency_map(gd))    
    
    frame_saliency_maps = np.array(frame_saliency_maps)
    
    # apply normalization to get the information of the relation with other heatmaps
    frame_saliency_maps = normalize(frame_saliency_maps)
    
    # apply some threashold to remove usesell peak
    frame_saliency_maps[frame_saliency_maps<=.2] = 0
    return frame_saliency_maps

##
## PLOT FUNCTIONS
##

def create_axes(sub=(1,1)):
    
    dpi = 100
    ncols=sub[0]
    nrows=sub[1]
    # determine the figure size in inches
    figsize = (1280*ncols/dpi, 800*nrows/dpi)
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize, dpi=dpi)
    
    extent = [0, DISPLAY_SIZE[0], DISPLAY_SIZE[1], 0]

    if not isinstance(axes, type(np.array)): 
        axes = np.array(axes) 
    
    for ax in axes.flatten(): 
        ax.axis(extent)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    
    return axes.flatten()

def plot_images(axes, frames):
    images = get_media_frame(media, frames)
    for ax, im in zip(axes, images):
        ax.imshow(im, origin='lower')
    
def plot_coordinates(axes, data, frames, retax=False):
    frame_coordinates = get_frame_coordinates(data, frames)
    for ax, coord in zip(axes, frame_coordinates):
        for c in coord:
            x, y, _, _ = c
            ax.scatter(x, y, s=10, alpha=.8, c='cyan')

def plot_fixations(axes, data, frames, retax=False):
    frame_fixations = get_frame_fixations(data, frames)
    for ax, fix in zip(axes, frame_fixations):
        for f in fix:
            x, y, d, distance, _ = f
            ax.scatter(x, y, s=2, alpha=.8, c='orange')
            circle = plt.Circle((x, y), 1e-2*d, alpha=.8, color='orange', fill=False)
            ax.add_artist(circle)
    if retax:
        return ax
        
def plot_saliency_map(axes, data, frames):
    frame_saliency_maps = get_frame_saliency_maps(data, frames)
    for ax, heatmap in zip(axes, frame_saliency_maps):
        ax.imshow(heatmap, extent=extent, cmap=cm_jet, vmin=0, vmax=1, alpha=.8)

def plot_emotion_saliency_map(axes, data, frames):
    frame_saliency_maps = get_frame_emotion_saliency_maps(data, frames)
    for ax, heatmap in zip(axes, frame_saliency_maps):
        ax.imshow(heatmap, extent=extent, cmap=cm_jet, vmin=0, vmax=1, alpha=.8)

In [3]:
mediafile = 53
media, coordinates, fixations = get_data(mediafile)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


FileNotFoundError: File b'sm_data_coord_53.csv' does not exist

In [ ]:
frames = [100]
col = 1
#axes = create_axes((col,len(frames)//col))
#plot_images(axes, frames)
#plot_coordinates(axes, coordinates, frames)

In [ ]:
axes = create_axes((col,len(frames)//col))
plot_images(axes, frames)
plot_fixations(axes, fixations, frames)

In [ ]:
axes = create_axes((col,len(frames)//col))
plot_images(axes, frames)
plot_saliency_map(axes, fixations, frames)

In [ ]:
axes = create_axes((col,len(frames)//col))
plot_images(axes, frames)
plot_emotion_saliency_map(axes, fixations, frames)